In [ ]:
import numpy as np 
import pandas as pd

Resuming download from 34603008 bytes (286113780 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/drgfreeman/rockpaperscissors?dataset_version_number=2 (34603008/320716788) bytes left.


100%|██████████| 306M/306M [00:20<00:00, 13.9MB/s] 

Extracting files...


Path to dataset files: C:\Users\micha\.cache\kagglehub\datasets\drgfreeman\rockpaperscissors\versions\2
